# Prepare Cell Line Metadata 

- **License:** [MIT License](https://opensource.org/licenses/MIT)
- **Version:** 0.2
- **Edit Log:** 
    - 2024-01-19: Initial version of the notebook
    - 2024-02-21: Revised the notes 


**Notebook Summary:** 

In this notebook, I will prepare the cell line metadata with enhancements and additional information to analyze this dataset. I will also create smaller helper data here, such as Reactome pathway hierarchy reference and detailed protein info data from UniProt.  

To expand the cell line information and create comprehensive metadata, I've done the following steps:
1. Grab the model annotation data from the [DepMap Models](https://depmap.org/portal/download/)
2. Merge with the model annotation and add the following information:
    - sample site
    - cancer type detail
    - tissue status
    - age 
    - Sex 
3. Due to some of those information is left unknown create a manual curation to fill those especially the age.
4. Adding a higher variable called "System Name" indicating the system that the cell line is originated from. The Tissue to System mapper is created from the ["National Cancer Institute"(NCI)](https://www.cancer.gov/types/by-body-location)
5. Also removed cell lines that have missing annotation in Age, Tumour Status (Primary, Metastatic), Sex, etc. 
6. Removed some cell lines from the lymphoid system since there are a lot of diversity in the subtypes which might skew the results in the favour of lymphoid system.

---

**Data Information:**

The data used in this notebook and this part of the analysis is from [Gonçalves et al. 2022](https://www.cell.com/cancer-cell/fulltext/S1535-6108(22)00274-4) and the data can be downloaded from [figshare Repo](https://figshare.com/articles/dataset/Pan-cancer_proteomic_map_of_949_human_cell_lines/19345397). For this notebook I am going to be using the following files accessed on 2023-11-01: 
- `ProCan-DepMapSanger_mapping_file_replicates.txt` - Containing a metadata of the samples at a replicate level, I've saved this as `ProCan-DepMapSanger_Metadata.txt` for my own convenience in the `./data/raw/` folder. 

Additionally I am also going to be using the following data from the [DepMap Portal](https://depmap.org/portal/download/) accessed on 2023-11-01:
- `Model_v2.csv` - Containing a cell line model information in depth, I've saved this as `DepMap_Model_Annotations.csv` for my own convenience in the `./data/raw/` folder.

For the manual additions and curations I've made the following files and place them in the `./data/raw/` folder:
- `Tissue_System_Map.csv` - Contains the mapping of tissues indicated in the depMap data to the systems for higher level grouping.
- `manuscript_cell_line_annotations-manual.csv` - This is the manually updated annotations for the cell lines that have missing information in the depMap data.

---

For the reactome pathway hierarchy helper dataset I will be using the following data from the [Reactome](https://reactome.org/download-data) accessed on 2023-11-01:
- `ReactomePathways.txt` - Containing all pathways available in the database
- `ReactomePathwaysRelation.txt` - Containing a pathway hierarchy relationships eg. col1 child, col2 parent
- Both of them saved as is in the `./data/raw/` folder.

---

For the fasta mapper, I've downloaded human "reviewed proteome without isoforms" fasta from [Uniprot](https://www.uniprot.org/) accessed on 2023-11-01 and saved it in the `./data/raw/` folder as `HUMAN_REVIEWED_2023_11_01.fasta`. 


## Setup Notebook

This part is a standard for my notebooks, where I import the all used libraries, set the design of the notebook, and define the paths for data and figures. 

> **Note:** The HTML rendering of this notebook will not show the code cells by default, but the code can be visible by clicking the "code" buttons aligned on the right.

### Libraries Used

In [1]:
import sys
import warnings

import pandas as pd

sys.path.append('../')          # Expand the path to parent dir
from questvar import utils      # Import custom modules
 
warnings.filterwarnings('ignore')
# Initialize the timer
startTime = utils.getTime()

### Notebook's Design

In [2]:
## Configure dataframe displaying
pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 25)

### Data Paths

I keep the data and figures folders seperate to allow for a more organized structure. Also within the data folder I create raw, processed, and in later notebook results folder to seperate each step of the process. 

The raw `data/raw/` folder needs to be present since it should contain the original data. For only this notebook I will save the expanded metadata and other helper data in the `data/raw/` folder as well since I consider this notebook to be pre-organizational step.

In [3]:
# Establish the paths
data_path = "./data/"
input_path = f"{data_path}raw/"

# Expand the Existing Metadata

The metadata and cell line information in the manuscript's data folder didn't have certain atributes and had some missing information which prompted to do some expansion with depMap model informations and manual curations. 

## Sample Metadata from the Manuscript

This is the replicate level metadata from the manuscript, containing the "Automatic_MS_filename" as the column names, and other information such as batch info, cell line id, tissue, cancer, and subtype infos.

In [4]:
replica_meta = pd.read_csv(
    input_path+'ProCan-DepMapSanger_Metadata.txt', sep="\t"
).sort_values([
    "Cell_line", "Batch", "Instrument", "Date"
]).reset_index(drop=True)
replica_meta["Date"] = replica_meta["Automatic_MS_filename"].str.split('_').str[0]
print("Data shape:", replica_meta.shape)
replica_meta.head()

Data shape: (6864, 10)


,Automatic_MS_filename,Batch,Date,Instrument,Cell_line,SIDM,Project_Identifier,Tissue_type,Cancer_type,Cancer_subtype
0,191005_b43-t1-13_00dlf_00jcq_m01_s_1,P04,191005,M01,201T,SIDM00055,SIDM00055;201T,Lung,Non-Small Cell Lung Carcinoma,Lung Adenocarcinoma
1,191006_b45-t1-13_00dmb_00jd7_m01_s_1,P04,191006,M01,201T,SIDM00055,SIDM00055;201T,Lung,Non-Small Cell Lung Carcinoma,Lung Adenocarcinoma
2,191008_b47-t1-13_00dn7_00jfo_m01_s_1,P04,191008,M01,201T,SIDM00055,SIDM00055;201T,Lung,Non-Small Cell Lung Carcinoma,Lung Adenocarcinoma
3,191008_b45-t2-13_00dmb_00jep_m03_s_1,P04,191008,M03,201T,SIDM00055,SIDM00055;201T,Lung,Non-Small Cell Lung Carcinoma,Lung Adenocarcinoma
4,191009_b43-t2-13_00dlf_00jg8_m03_s_1,P04,191009,M03,201T,SIDM00055,SIDM00055;201T,Lung,Non-Small Cell Lung Carcinoma,Lung Adenocarcinoma


## Updating the Metadata with Simple Navigation Columns

I add the following columns to the metadata to allow for easier navigation and filtering in the future.

- **VarianceID** - This is the unique batch identifier put together by joining "Instrument" and "Batch" columns.
- **Replicate** - Replicate number indicator per VarianceID
- **UniqueSample** - Unique sample identifier per cell line, this is created by joining the "VarianceID" and "SIDM" columns. (SIDM is the cell line model id)
- **ColumnName** - New simpler column names used by combining "UniqueSample" and "Replicate" columns. This is used to make the column names shorter and easier to read.

Then I select the columns that I want to keep; `["Automatic_MS_filename", "ColumnName", "UniqueSample", "VarianceID", "Cell_line", "Replicate", "SIDM"]`

Finally I drop the `Control` samples from the cell lines since I want to focus on only the cancer cell lines.

In [5]:
# Create variance ID to hold Instrument and Batch info as one
replica_meta["VarianceID"] = replica_meta.apply(
    lambda x: "|".join([x["Instrument"], x["Batch"]]), 
    axis=1
)
# Create Replica info for each sample belong to same cell line - variance ID combo
replica_meta["Replicate"] = "1"
for n, g in replica_meta.groupby(["VarianceID", "SIDM"]):
    if g.shape[0] > 1:
        replica_meta.loc[
            (replica_meta["VarianceID"]==n[0]) & 
            (replica_meta["SIDM"]==n[1]), 
            "Replicate"
        ] = [str(i+1) for i in range(g.shape[0])]
# Create UniqueSample column indicating cell line - variance id
replica_meta["UniqueSample"] = replica_meta.apply(
    lambda x: "|".join([x["VarianceID"], x["SIDM"]]), 
    axis=1
)
# Create Unique column name using uniqueSample and replicate
replica_meta["ColumnName"] = replica_meta.apply(
    lambda x: "|".join([x["UniqueSample"], x["Replicate"]]), 
    axis=1
)
# Remove columns that are not gonna be used
replica_meta = replica_meta[[
    "Automatic_MS_filename", "ColumnName", "UniqueSample", 
    "VarianceID", "Cell_line", "Replicate", "SIDM"
]]

# Remove the Control_HEK293T samples
replica_meta = replica_meta[replica_meta["SIDM"] != 'Control_HEK293T']
# Preview the results
print("Data shape:", replica_meta.shape)
replica_meta.head()

Data shape: (5800, 7)


,Automatic_MS_filename,ColumnName,UniqueSample,VarianceID,Cell_line,Replicate,SIDM
0,191005_b43-t1-13_00dlf_00jcq_m01_s_1,M01|P04|SIDM00055|1,M01|P04|SIDM00055,M01|P04,201T,1,SIDM00055
1,191006_b45-t1-13_00dmb_00jd7_m01_s_1,M01|P04|SIDM00055|2,M01|P04|SIDM00055,M01|P04,201T,2,SIDM00055
2,191008_b47-t1-13_00dn7_00jfo_m01_s_1,M01|P04|SIDM00055|3,M01|P04|SIDM00055,M01|P04,201T,3,SIDM00055
3,191008_b45-t2-13_00dmb_00jep_m03_s_1,M03|P04|SIDM00055|1,M03|P04|SIDM00055,M03|P04,201T,1,SIDM00055
4,191009_b43-t2-13_00dlf_00jg8_m03_s_1,M03|P04|SIDM00055|2,M03|P04|SIDM00055,M03|P04,201T,2,SIDM00055


## Open and Clean DepMap Models Annotations

DepMap Models annotation data contains a lot of information about the cell lines, but I am only interested in the following columns:
- **SangerModelID** - Cell line model id (used to join with the manuscript's metadata)
- **CellLineName** - Cell line name (used to join with the manuscript's metadata)
- **AgeCategory** - Age category of the cell line
- **PrimaryOrMetastasis** - Tumour status of the cell line
- **Sex** - The sex of the patient that the cell line is derived from
- **SampleCollectionSite** - The site where the sample is collected from
- **OncotreeLineage** - The lineage of the cell line
- **OncotreePrimaryDisease** - The primary disease of the cell line
- **OncotreeSubtype** - The subtype of the cell line

I am particularly interested in the `AgeCategory`, `PrimaryOrMetastasis`, `Sex`, `SampleCollectionSite`, `OncotreeLineage`, `OncotreePrimaryDisease`, and `OncotreeSubtype` columns, which can be used in various groupings when testing and summarizing the data.

Using DepMap Models annotation data also changes the tissue names for the cell line origins for certain cancers for instance, originally myleoid and lymphoid cancers are all grouped under the Hematopoietic and Lymphoid Tissue but with the DepMap Models annotation data I can separate them into Myeloid and Lymphoid systems. 

In [6]:
depmap_info= pd.read_csv(
    input_path+"DepMap_Model_Annotations.csv",
)[[
    "SangerModelID",
    "CellLineName",
    "AgeCategory",
    "PrimaryOrMetastasis",
    # "GrowthPattern",
    "Sex",
    "SampleCollectionSite",
    # "CCLEName", 
    "OncotreeLineage",
    "OncotreePrimaryDisease",
    "OncotreeSubtype",
]].set_index("SangerModelID")
print("Number of Models in DepMap:", depmap_info.shape[0])

# Get the Unique Cell Ids from paper
paper_cell_ids = set(replica_meta["SIDM"])
print("Unmatched cell line ids:", (paper_cell_ids - set(depmap_info.index)))
# # Only select cell lines that are in the paper
depmap_info = depmap_info.loc[
    list(set.intersection(paper_cell_ids, depmap_info.index))
]

# Add unmatched cell lines to the depmap info with NA values
depmap_info = pd.concat([
    depmap_info,
    pd.DataFrame(
        index=list(paper_cell_ids - set(depmap_info.index)),
        columns=depmap_info.columns
    )
])

# Replace the cell line names in the depmap info with the paper's cell line names
# Merge the cell line names from the paper to the depmap info
depmap_info = depmap_info.merge(
    replica_meta[[
        "SIDM", "Cell_line"
    ]].drop_duplicates().set_index("SIDM"),
    left_index=True, 
    right_index=True, 
    how="left"
).drop(columns=["CellLineName"])

print("Number of Cell Line info in DepMap:", depmap_info.shape[0])
depmap_info.head()

Number of Models in DepMap: 1864
Unmatched cell line ids: {'SIDM01117', 'SIDM00427'}
Number of Cell Line info in DepMap: 950


,AgeCategory,PrimaryOrMetastasis,Sex,SampleCollectionSite,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,Cell_line
SIDM00018,Adult,NaN,Male,bone_marrow,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,K052
SIDM00023,Unknown,NaN,Unknown,oesophagus,Esophagus/Stomach,Esophageal Squamous Cell Carcinoma,Esophageal Squamous Cell Carcinoma,TE-12
SIDM00040,Adult,Metastatic,Male,lymph_node,Esophagus/Stomach,Esophagogastric Adenocarcinoma,Stomach Adenocarcinoma,TMK-1
SIDM00041,Unknown,NaN,Unknown,soft_tissue,Esophagus/Stomach,Esophagogastric Adenocarcinoma,Stomach Adenocarcinoma,STS-0421
SIDM00042,Unknown,Primary,Unknown,pancreas,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PL4


After cleaning the data and selecting cell line models that are present in the manuscript now I have a data that contains the cell line information. However this has a lot of Unknown and NaN values, which might be updated by manual check-up and curation.

## Adding System Names to DepMap Models Annotations

One additional information that I want to add to the metadata is the system name of the cell line, which is the higher level grouping of the tissue. I've created a mapper from the tissue names in the DepMap Models annotation data to the system names using the ["National Cancer Institute"(NCI)](https://www.cancer.gov/types/by-body-location) as the reference. The `Tissue_System_Map.csv` file contains the mapping of the tissues indicated in the depMap data to the systems for higher level grouping.

In [7]:
mapper = pd.read_csv(
    input_path+"Tissue_System_Map.csv"
)
mapper.head()

# Map the tissue type to system
depmap_info = depmap_info.reset_index().merge(
    mapper, 
    how="left", 
    left_on="OncotreeLineage", 
    right_on="Tissue"
).drop(
    "Tissue", axis=1
).rename(
    columns={"System": "System_name"}
)
print("Number of Cell Line info in DepMap:", depmap_info.shape[0])
depmap_info.head()

Number of Cell Line info in DepMap: 950


,index,AgeCategory,PrimaryOrMetastasis,Sex,SampleCollectionSite,OncotreeLineage,OncotreePrimaryDisease,OncotreeSubtype,Cell_line,System_name
0,SIDM00018,Adult,NaN,Male,bone_marrow,Myeloid,Acute Myeloid Leukemia,Acute Myeloid Leukemia,K052,Hematologic
1,SIDM00023,Unknown,NaN,Unknown,oesophagus,Esophagus/Stomach,Esophageal Squamous Cell Carcinoma,Esophageal Squamous Cell Carcinoma,TE-12,Digestive
2,SIDM00040,Adult,Metastatic,Male,lymph_node,Esophagus/Stomach,Esophagogastric Adenocarcinoma,Stomach Adenocarcinoma,TMK-1,Digestive
3,SIDM00041,Unknown,NaN,Unknown,soft_tissue,Esophagus/Stomach,Esophagogastric Adenocarcinoma,Stomach Adenocarcinoma,STS-0421,Digestive
4,SIDM00042,Unknown,Primary,Unknown,pancreas,Pancreas,Pancreatic Adenocarcinoma,Pancreatic Adenocarcinoma,PL4,Digestive


## Manual Curation of the Metadata

### Clean and Save Annotations for Manual Editing (Filling Gaps)

I save the cell line info table to raw data folder as `manuscript_cell_line_annotations.csv` for manual editing. Which I've already done and created the `manuscript_cell_line_annotations-manual.csv` file. But I am keepnig this here to highlight the process.

THe saved file has simpliefied column names and only contains the cell line models that are present in the manuscript.

In [8]:
# Clean-up and save the data
depmap_info.columns = [
    "SIDM",
    "Age",
    "Status",
    "Sex",
    "Collection_site",
    "Tissue_type",
    "Cancer_type",
    "Cancer_subtype",
    "Cell_line",
    "System_name"
]

# Order the columns
depmap_info = depmap_info[[
    "SIDM",
    "Cell_line",
    "Sex",
    "Age",
    "Status",
    "Collection_site",
    "System_name",
    "Tissue_type",
    "Cancer_type",
    "Cancer_subtype"
]].sort_values([
    "System_name",
    "Tissue_type",
    "Cancer_type",
    "Cancer_subtype",
]).reset_index(drop=True)

# Save the data
depmap_info.to_csv(
    input_path + "manuscript_cell_line_annotations.csv",
    index=False
)

# Preview the data
depmap_info.head()

,SIDM,Cell_line,Sex,Age,Status,Collection_site,System_name,Tissue_type,Cancer_type,Cancer_subtype
0,SIDM00772,HCC2218,Female,Adult,Primary,breast,Breast,Breast,Breast Ductal Carcinoma In Situ,Breast Ductal Carcinoma In Situ
1,SIDM00872,HCC1954,Female,Adult,Primary,breast,Breast,Breast,Breast Ductal Carcinoma In Situ,Breast Ductal Carcinoma In Situ
2,SIDM00875,HCC1806,Female,Adult,Primary,breast,Breast,Breast,Breast Ductal Carcinoma In Situ,Breast Ductal Carcinoma In Situ
3,SIDM00879,HCC1500,Female,Adult,Primary,breast,Breast,Breast,Breast Ductal Carcinoma In Situ,Breast Ductal Carcinoma In Situ
4,SIDM00954,COLO-824,Female,Adult,Metastatic,pleural_effusion,Breast,Breast,"Breast Neoplasm, NOS","Breast Neoplasm, NOS"


### Manual Annotation

In the manual annotation I use multiple sources to fill the missing information in the metadata. I mainly used the [Cellosaurus](https://www.cellosaurus.org/) resource as well as other google to see if any information can be found. Overall I was able to fill the missing information about half of the times. And the ones I wasn't able to fill I left them as Unknown or NaN.

Also I've created a column called "Drop" while I was doing the manual curation, if the value inside is 1 then I will drop that cell line when I am done with the manual curation.

### Clean the Manual Annotations

After opening the manually annotated metadata I clean the data by removing the cell lines that have missing info labeled with Drop = 1. Also I've removed the cell lines with Unknown Sex and Unknown or Fetus Age since I want to focus on Adult and Childhood cancers. 

In [9]:
cell_info_cur = pd.read_csv(input_path+"manuscript_cell_line_annotations-manual.csv")
print("Data Shape:", cell_info_cur.shape)
# Remove the cell lines with drop = 1
cell_info_cur = cell_info_cur[cell_info_cur["Drop"]!=1]
cell_info_cur = cell_info_cur.drop(columns="Drop").reset_index(drop=True)

# Remove cell lines (Sex = Unknown & Age = [Unknown, Fetus])
cell_info_cur = cell_info_cur[
    (cell_info_cur["Sex"]!="Unknown") & 
    (~cell_info_cur["Age"].isin(["Unknown", "Fetus"]))
]

cell_info_cur["Age"] = cell_info_cur["Age"].replace("Pediatric", "Child")

cell_info_cur.sort_values([
    "System_name",
    "Tissue_type",
    "Cancer_type",
    "Cancer_subtype",
    "Status",
    "Age",
]).reset_index(drop=True)

print("Data after manually labeled for dropping is done:", cell_info_cur.shape)
cell_info_cur.head()

Data Shape: (950, 11)
Data after manually labeled for dropping is done: (814, 10)


,SIDM,Cell_line,Sex,Age,Status,Collection_site,System_name,Tissue_type,Cancer_type,Cancer_subtype
0,SIDM00287,BALL-1,Male,Adult,Primary,Peripheral_blood,Hematologic,Lymphoid,Acute Lymphoblastic Leukemia,Adult B-Lymphoblastic Leukemia
1,SIDM00429,NALM-6,Male,Adult,Primary,Peripheral_blood,Hematologic,Lymphoid,Acute Lymphoblastic Leukemia,Adult B-Lymphoblastic Leukemia
2,SIDM00813,RPMI-6666,Male,Adult,Primary,Peripheral_blood,Hematologic,Lymphoid,Acute Lymphoblastic Leukemia,Adult B-Lymphoblastic Leukemia
3,SIDM01086,RS4-11,Female,Adult,Primary,Bone_marrow,Hematologic,Lymphoid,Acute Lymphoblastic Leukemia,Adult B-Lymphoblastic Leukemia
4,SIDM00415,ROS-50,Male,Adult,Primary,Peripheral_blood,Hematologic,Lymphoid,Acute Lymphoblastic Leukemia,Adult B-Lymphoblastic Leukemia


After the dropping some of the cell line based on the criterias above from 950 cell lines I am left with 814 cell lines.

## Replica and Cell Line Metadata is Ready

Since we dropped some cell lines in the manual curation step, I will up date the replica metadata as well and ensure they are in sync with the cell line metadata.

In [10]:
print("Replicate data before:", replica_meta.shape[0])
replica_meta = replica_meta.set_index("SIDM").loc[list(set(cell_info_cur["SIDM"]))]
print("Replicate data after:", replica_meta.shape[0])
replica_meta.head()

Replicate data before: 5800
Replicate data after: 4978


,Automatic_MS_filename,ColumnName,UniqueSample,VarianceID,Cell_line,Replicate
SIDM,,,,,,
SIDM00183,200117_b1-8-t1-1_00q2c_00qi2_m01_s_1,M01|P06|SIDM00183|1,M01|P06|SIDM00183,M01|P06,LB647-SCLC,1
SIDM00183,200130_b8-1-t2-1_00q59_00rp7_m01_s_1,M01|P06|SIDM00183|2,M01|P06|SIDM00183,M01|P06,LB647-SCLC,2
SIDM00183,200118_b1-8-t2-1_00q2c_00qim_m03_s_1,M03|P06|SIDM00183|1,M03|P06|SIDM00183,M03|P06,LB647-SCLC,1
SIDM00183,200121_b3-8-t2-1_00q38_00qko_m03_s_1,M03|P06|SIDM00183|2,M03|P06|SIDM00183,M03|P06,LB647-SCLC,2
SIDM00183,200129_b8-1-t1-1_00q59_00roe_m05_s_1,M05|P06|SIDM00183|1,M05|P06|SIDM00183,M05|P06,LB647-SCLC,1


### Save the Final Data

The cleaned replicate metadata and expanded cell line metadata is saved in the `data/raw/` folder as `cleaned_replica_metadata.csv` and `final_cell_line_annotation.csv` respectively.

In [11]:
cell_info_cur.to_csv(input_path+"final_cell_line_annotation.csv", index=False)
replica_meta.to_csv(input_path+"cleaned_replica_metadata.csv")

# Helper Data Tables






## UniProt Protein Information

Having access to the expanded protein information is useful when reporting the results with more than just protien accession ids. I've downloaded the human "reviewed proteome without isoforms" fasta from [Uniprot](https://www.uniprot.org/) accessed on 2023-11-01 and saved it in the `./data/raw/` folder as `HUMAN_REVIEWED_2023_11_01.fasta`.

The using the `fasta_to_df` function from `utils.py` script I've parsed the fasta file and created a dataframe: 

In [12]:
fasta_data = utils.fasta_to_df(
    f"{input_path}HUMAN_REVIEWED_2023_11_01.fasta",
    fasta_ID="" # This is not needed for this usecase
).drop(
    columns=[
        "fastaId", 
        "reviewStatus", 
        "isoformStatus", 
        "molecularWeight_kDa",
        "sequence",
    ]
)
# # Save the data as feather
# feather.write_dataframe(
#     fasta_data, 
#     input_path+"UniProtIDMapping.feather"
# )
fasta_data.to_csv(
    input_path+"UniProtIDMapping.csv", 
    index=False
)

print("Data Shape:", fasta_data.shape)
fasta_data.head()

Data Shape: (20265, 5)


,entry,entryName,geneName,proteinDescription,sequenceLength
0,A0A024R1R8,TMA7B_HUMAN,TMA7B,Translation machinery-associated protein 7B,64
1,A0A024RBG1,NUD4B_HUMAN,NUDT4B,Diphosphoinositol polyphosphate phosphohydrola...,181
2,A0A075B6H7,KV37_HUMAN,IGKV3-7,Probable non-functional immunoglobulin kappa v...,116
3,A0A075B6H8,KVD42_HUMAN,IGKV1D-42,Probable non-functional immunoglobulin kappa v...,117
4,A0A075B6H9,LV469_HUMAN,IGLV4-69,Immunoglobulin lambda variable 4-69,119


Resulting dataframe is saved in the `./data/raw/` folder as `UniProtIDMapping.csv`, I opted to save it as csv since it is easier to read and edit.

## Reactome Pathway Hierarchy

The reactome pathway hierarchy is not directly available when I use enrichment analysis results from `g:Profiler` that's why I want to createa a table that enables a quick lookup of the pathway hierarchy as well as a mechanism to group pathways into highest-level parent pathways.

The reactome pathway hierarchy is downloaded from the [Reactome](https://reactome.org/download-data) accessed on 2023-11-01 and saved in the `./data/raw/` folder as `ReactomePathways.txt` and `ReactomePathwaysRelation.txt`.

In [13]:
reactome_pathways = pd.read_csv(
    f"{input_path}ReactomePathways.txt", sep="\t", header=None
)
print()
reactome_pathways.columns = ["PathwayID", "PathwayName", "Organism"]
print("Data Shape:", reactome_pathways.shape)
utils.print_series(
    reactome_pathways["Organism"].value_counts(),
    header="Number of pathways per organism"
)
print()
# Subset for only human pathways
reactome_pathways = reactome_pathways[
    reactome_pathways["Organism"] == "Homo sapiens"
].reset_index(drop=True)
print("Data Shape after:", reactome_pathways.shape)
reactome_pathways.head()


Data Shape: (22084, 3)
Number of pathways per organism
 Homo sapiens -> 2673
 Mus musculus -> 1737
 Gallus gallus -> 1736
 Rattus norvegicus -> 1724
 Bos taurus -> 1718
 Sus scrofa -> 1709
 Canis familiaris -> 1690
 Drosophila melanogaster -> 1511
 Xenopus tropicalis -> 1502
 Danio rerio -> 1353
 Caenorhabditis elegans -> 1350
 Dictyostelium discoideum -> 1024
 Schizosaccharomyces pombe -> 864
 Saccharomyces cerevisiae -> 850
 Plasmodium falciparum -> 630
 Mycobacterium tuberculosis -> 13

Data Shape after: (2673, 3)


,PathwayID,PathwayName,Organism
0,R-HSA-164843,2-LTR circle formation,Homo sapiens
1,R-HSA-73843,5-Phosphoribose 1-diphosphate biosynthesis,Homo sapiens
2,R-HSA-1971475,A tetrasaccharide linker sequence is required ...,Homo sapiens
3,R-HSA-5619084,ABC transporter disorders,Homo sapiens
4,R-HSA-1369062,ABC transporters in lipid homeostasis,Homo sapiens


For the purposes of this analysis I am only interested in the human pathways, so I filter the data to only contain the human pathways. Then I open the hierarcy data and create a dictionary that contains the pathway hierarchy. and I make sure to only get the hierarcy for the human pathways.


In [14]:
paired_hierarchy = pd.read_csv(
    f"{input_path}ReactomePathwaysRelation.txt",
    sep="\t", 
    header=None
)
paired_hierarchy.columns = ["Parent", "Child"]

# Subset only Human pathways in paired_hierarcy
paired_hierarchy = paired_hierarchy[
    (paired_hierarchy["Parent"].isin(reactome_pathways["PathwayID"])) | 
    (paired_hierarchy["Child"].isin(reactome_pathways["PathwayID"]))
].reset_index(drop=True)

print("Data Shape:", paired_hierarchy.shape)
paired_hierarchy.head()

Data Shape: (2665, 2)


,Parent,Child
0,R-HSA-109581,R-HSA-109606
1,R-HSA-109581,R-HSA-169911
2,R-HSA-109581,R-HSA-5357769
3,R-HSA-109581,R-HSA-75153
4,R-HSA-109582,R-HSA-140877


To facilitate the lookup of the pathway hierarchy I merge the parent pathway information to the main reactome pathway data table. As an additional information I create a column called "NumChildren" which will indicate how many iterative children pathway are there for the given pathway. If the pathway is at the very bottom of the hierarchy then the value will be 0.

In [15]:
def count_children(term, term_dict):
    # Base case: The term is not in the dictionary (it has no children)
    if term not in term_dict:
        return 0

    # Recursive case: The term is in the dictionary
    else:
        children = term_dict[term]
        count = len(children)  # Count the direct children
        for child in children:
            count += count_children(child, term_dict)  # Add the count of the nested children
        return count

# Identify the highest hierarchy Parents
# Find Parent ids that are not in Child
mainParent_IDs = list(
    set(paired_hierarchy["Parent"].tolist()) - 
    set(paired_hierarchy["Child"].tolist())
)
print("The number of mainParentIDs:", len(mainParent_IDs))

# Create Parent: All Children Dictionary
term_dict = paired_hierarchy.groupby('Parent')['Child'].apply(list).to_dict()
term_name_dict = reactome_pathways.set_index("PathwayID")["PathwayName"].to_dict()
# Add the Parent column to the reactome_pathways dataframe
reactome_pathways["Parent"] = reactome_pathways["PathwayID"].map(
    paired_hierarchy.set_index("Child")["Parent"].to_dict()
)
# Add the number of children to the reactome_pathways dataframe
reactome_pathways["NumChildren"] = reactome_pathways["PathwayID"].apply(
    lambda x: count_children(x, term_dict)
)

# Save the data
reactome_pathways[[
    "PathwayID", "PathwayName", "Parent", "NumChildren"
]].to_csv(
    f"{input_path}ReactomeDetailed.txt", 
    sep="\t", 
    index=False
)

reactome_pathways.head()

The number of mainParentIDs: 29


,PathwayID,PathwayName,Organism,Parent,NumChildren
0,R-HSA-164843,2-LTR circle formation,Homo sapiens,R-HSA-162592,0
1,R-HSA-73843,5-Phosphoribose 1-diphosphate biosynthesis,Homo sapiens,R-HSA-71336,0
2,R-HSA-1971475,A tetrasaccharide linker sequence is required ...,Homo sapiens,R-HSA-1793185,0
3,R-HSA-5619084,ABC transporter disorders,Homo sapiens,R-HSA-5619115,15
4,R-HSA-1369062,ABC transporters in lipid homeostasis,Homo sapiens,R-HSA-382556,0


The resulting dataframe is saved in the `./data/raw/` folder as `ReactomeDetailed.txt`, I opted to save it as a txt file since the this data won't be needed to be opened frequently. 

---

Note: Not related to the creating of the data, but I've also created a function called list_children which neatly prints the hierarchy of the given pathway. Here are some examples of how it looks like:

In [16]:
def list_children(term, term_dict, term_name_dict):
    """Create a neat new line and tab listed print of the children of a term"""
    # Base case: The term is not in the dictionary (it has no children)
    if term not in term_dict:
        return ""

    # Recursive case: The term is in the dictionary
    else:
        children = term_dict[term]
        children_names = [term_name_dict[child] for child in children]
        children_names.sort()
        children_names = "\n".join(children_names)
        children_names = "\n\t" + children_names
        for child in children:
            children_names += list_children(child, term_dict, term_name_dict)
        return children_names
    
term = "R-HSA-75072" 
print(term ,"-" , term_name_dict[term])
print(list_children(term, term_dict, term_name_dict))

R-HSA-75072 - mRNA Editing

	mRNA Editing: A to I Conversion
mRNA Editing: C to U Conversion
	Formation of the Editosome
	C6 deamination of adenosine
Formation of editosomes by ADAR proteins


In [17]:
term = "R-HSA-72306"
print(term ,"-" , term_name_dict[term])
print(list_children(term, term_dict, term_name_dict))

R-HSA-72306 - tRNA processing

	tRNA modification in the mitochondrion
tRNA modification in the nucleus and cytosol
tRNA processing in the mitochondrion
tRNA processing in the nucleus
	Synthesis of wybutosine at G37 of tRNA(Phe)


# Conclusion

This notebook is the first step in the analysis of the cancer cell line data. In this notebook I've prepared the a comprehensive metadata for the cell lines that will be used in the analysis and other useful helper tables for reactome hierarchies and uniprot protein information.

In [18]:
print("Notebook Execution Time:", utils.prettyTimer(utils.getTime() - startTime))

Notebook Execution Time: 00h:00m:02s
